In [5]:
#write your names and students ids here

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from gensim.models import Word2Vec, KeyedVectors



# Install spaCy (run in terminal/prompt)
import sys
#!{sys.executable} -m pip install spacy
# Download spaCy's  'en' Model
#!{sys.executable} -m spacy download en



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\UX550VD\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\UX550VD\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [49]:
data=pd.read_json ('../News_Category_Dataset_v2.json', lines=True)

### Typical data analysis of a dataframe now

In [ ]:
data.shape


In [ ]:
data.info()

In [ ]:
#data.describe
data.head()

In [ ]:
print(data.groupby('category').size())

In [ ]:
#we can see that many categories are similar.
#Let´s merge them. We will merge the more specific ones into the more general ones in case of doubt
#the code in the cell below was obtained from https://www.kaggle.com/rmisra/news-category-dataset/discussion/114275

In [10]:
data['category']=data['category'].replace({"HEALTHY LIVING": "WELLNESS",
"QUEER VOICES": "GROUPS VOICES",
"BUSINESS": "BUSINESS & FINANCES",
"PARENTS": "PARENTING",
"BLACK VOICES": "GROUPS VOICES",
"THE WORLDPOST": "WORLD NEWS",
"STYLE": "STYLE & BEAUTY",
"GREEN": "ENVIRONMENT",
"TASTE": "FOOD & DRINK",
"WORLDPOST": "WORLD NEWS",
"SCIENCE": "SCIENCE & TECH",
"TECH": "SCIENCE & TECH",
"MONEY": "BUSINESS & FINANCES",
"ARTS": "ARTS & CULTURE",
"COLLEGE": "EDUCATION",
"LATINO VOICES": "GROUPS VOICES",
"CULTURE & ARTS": "ARTS & CULTURE",
"FIFTY": "MISCELLANEOUS",
"GOOD NEWS": "MISCELLANEOUS"})

In [ ]:
print(data.groupby('category').size())

In [11]:
(data.groupby('category').size()).std()

7299.533828002569

As we may observe, the data is still highly imbalanced. Therefore further techniques will have to be applied to
make up for this

## Data cleaning:

We must  apply either lemmatization or stemming to the words.
We will try to go for lemmatization first as it depends on the context of the word.

We used the following link to guide ourselves https://www.machinelearningplus.com/nlp/lemmatization-examples-python/


//TODO: preproces words to remove apostrophes for example. isn´t -> is not

In [36]:
lemmatizer = WordNetLemmatizer()

def clean(text):

    # Define the sentence to be lemmatized
    text=text.lower() #making everything lowercase
    
    # Tokenize: Split the sentence into words
    word_list = nltk.word_tokenize(text)
    #print(word_list)

    # Lemmatize list of words and join
    #lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    #print(lemmatized_output)
    
   # return lemmatized_output
    return word_list



In [37]:

data['short_description'] = data['short_description'].apply(clean)
data['headline'] = data['headline'].apply(clean)


In [41]:
#the code from assignment 2 was changed here. We had a framework to start with this way.

class FeatureCreator:
    def __init__(self):
        self.voc = {}
        self.id = {}
        self.count = 0
    
    
    def createFeatureSet(self, data):
        # t1 and t2 are the 2 questions we want to compare
        for index, new in data.iterrows():
            
            for w in new['headline']:
                
                if w in self.voc:
                    self.voc[w] += 1
                else:
                    
                    self.voc[w] = 1
                    self.id[w] = self.count
                    self.count += 1;
                    
            for w in new['short_description']:
                if w in self.voc:
                    self.voc[w] += 1
                else:
                    self.voc[w] = 1
                    self.id[w] = self.count
                    self.count += 1;
                
        

class BowFeatureCreator(FeatureCreator):
    #still need to change this one
     def createFeatures(self,data):
        id_len=len(self.id) #we will add this length to the index if we want to access the second question. It is made clear in the implementation

        features = dok_matrix((len(data),id_len*2)) #we will have twice the entries. One |VOC| x |2| one for the headline and the other one for the description
        label = []
        for i in range(len(data)):
            for w in data[i][0]:
                if w in self.id:
                    features[i,self.id[w]] = 1
            for w in data[i][1]:
                if w in self.id:
                    features[i, self.id[w]+ id_len] = 1 #now there is one bow for the headline and another for the description
            label.append(data[i][2])
        return features,label
    

Calculate size of the vocabulary:

In [42]:
%%time
features=FeatureCreator()
features.createFeatureSet(data)
print(features.count)

117873
Wall time: 1min 8s


(89989808, 349124520)

KeyError: "Key 'Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song' not present"

In [43]:
.
.
.
.
.
..
.
.
.

.
.
.
.
..
.

.
.
.
.
.
.
.
..

.
.
.Ignore from here onwards

SyntaxError: invalid syntax (<ipython-input-43-e2636bdbbb7d>, line 1)

# IGNORE FROM HERE ONWARDS

//we will trin the model once we are done with the cleaning

First we split the data such that the proportion of category labels is the same in train and test. Remember, category will be our target

In [38]:
train, test = train_test_split(data, test_size=0.2, stratify=data['category'])

Leave that for later, do not pay too much attention for now

In [37]:

class_weights = dict(zip(np.unique(y_train), class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)))

Then you use the class weights during the training process:

train_history = model.fit( train_dataset, steps_per_epoch=n_steps, class_weight=class_weights )

SyntaxError: invalid syntax (<ipython-input-37-c27e5a755d4f>, line 3)

In [64]:
data.loc[1,'headline']

"Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song"

In [ ]:
sentences=[]
for i, row in data.iterrows():
    sentences.append(row['headline'])

model_embeddings=Word2Vec(vector_size=20, min_count=1)
model_embeddings.build_vocab(sentences, progress_per=10000)
model_embeddings.train(sentences, total_examples=model_embeddings.corpus_count, epochs=30,report_delay=1)
#model_embeddings.build(data['headlines'])


In [ ]:
model_embeddings.wv.most_similar(positive=["korea"])
model_embeddings.wv.most_similar(positive=["Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song"])